### Lab2 (a) Model preperation

In [1]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [3]:
# Load the best weight paramters
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9151


### Lab2 (b) Prune by percentage

In [4]:
def prune_by_percentage(layer, q=80.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # Convert the weight of "layer" to numpy array
    np_weight = layer.weight.detach().cpu().numpy()
    # Compute the q-th percentile of the abs of the converted array
    weight_abs = np.absolute(np_weight)
    qth_percentile = np.percentile(weight_abs, q)
    # Generate a binary mask same shape as weight to decide which element to prune
    binary_mask = weight_abs > qth_percentile
    # Convert mask to torch tensor and put on GPU
    mask_tensor = torch.from_numpy(binary_mask).float().to(device)
    # Multiply the weight by mask to perform pruning
    layer.weight.data = layer.weight.data * mask_tensor
    #print(layer.weight.data)
    #pass
    

In [5]:
net.load_state_dict(torch.load("pretrained_model.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # change q value
        #prune_by_percentage(layer, q = 40.0)
        #prune_by_percentage(layer, q = 60.0)
        prune_by_percentage(layer, q = 80.0)
        
        # Optional: Check the sparsity you achieve in each layer
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np_weight.size - np.count_nonzero(np_weight)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (c) Finetune pruned model

In [6]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element
            weight_mask[name] = layer.weight.data != 0.0
            #print(weight_mask[name])      
    
    
    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    layer.weight.data = layer.weight.data * weight_mask[name]
                    #print(layer.weight.data)
                    
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()
    
    

In [7]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=80.0)

# Training setup, do not change
batch_size=256
lr=0.002
reg=1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

==> Preparing data..
Files already downloaded and verified


C:\Users\Administrator\anaconda3\envs\pytorch\lib\site-packages\torch\utils\data\dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [10]:
# Model finetuning
for epoch in range(20): 
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")


Epoch: 0
[Step=50]	Loss=0.9905	acc=0.6847	540.0 examples/second
[Step=100]	Loss=0.8140	acc=0.7357	4869.5 examples/second
[Step=150]	Loss=0.7257	acc=0.7626	4852.8 examples/second
Test Loss=0.5564, Test acc=0.8182
Saving...

Epoch: 1
[Step=50]	Loss=0.4625	acc=0.8458	539.3 examples/second
[Step=100]	Loss=0.4588	acc=0.8461	4871.8 examples/second
[Step=150]	Loss=0.4538	acc=0.8464	4859.1 examples/second
Test Loss=0.4851, Test acc=0.8388
Saving...

Epoch: 2
[Step=50]	Loss=0.4149	acc=0.8596	542.1 examples/second
[Step=100]	Loss=0.4046	acc=0.8629	4857.9 examples/second
[Step=150]	Loss=0.3969	acc=0.8652	4851.6 examples/second
Test Loss=0.4536, Test acc=0.8506
Saving...

Epoch: 3
[Step=50]	Loss=0.3714	acc=0.8748	547.1 examples/second
[Step=100]	Loss=0.3704	acc=0.8744	4844.9 examples/second
[Step=150]	Loss=0.3644	acc=0.8747	4852.8 examples/second
Test Loss=0.4347, Test acc=0.8573
Saving...

Epoch: 4
[Step=50]	Loss=0.3479	acc=0.8787	543.3 examples/second
[Step=100]	Loss=0.3468	acc=0.8803	4848.0 ex

In [11]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np_weight.size - np.count_nonzero(np_weight)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (d) Iterative pruning

In [12]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):    
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch<10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                q = 8.0 * (epoch + 1)
                prune_by_percentage(layer, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0489	acc=0.9845	524.7 examples/second
[Step=100]	Loss=0.0481	acc=0.9849	4798.2 examples/second
[Step=150]	Loss=0.0482	acc=0.9847	4745.0 examples/second
Test Loss=0.3252, Test acc=0.9143

Epoch: 1
[Step=50]	Loss=0.0485	acc=0.9848	476.4 examples/second
[Step=100]	Loss=0.0487	acc=0.9838	4814.2 examples/second
[Step=150]	Loss=0.0499	acc=0.9835	4773.4 examples/second
Test Loss=0.3244, Test acc=0.9157

Epoch: 2
[Step=50]	Loss=0.0539	acc=0.9819	521.4 examples/second
[Step=100]	Loss=0.0519	acc=0.9831	4837.5 examples/second
[Step=150]	Loss=0.0531	acc=0.9823	4819.9 examples/second
Test Loss=0.3291, Test acc=0.9130

Epoch: 3
[Step=50]	Loss=0.0582	acc=0.9802	553.9 examples/second
[Step=100]	Loss=0.0565	acc=0.9812	4826.4 examples/second
[Step=150]	Loss=0.0572	acc=0.9809	4812.1 examples/second
Test Loss=0.3344, Test acc=0.9115

Epoch: 4
[Step=50]	Loss=0.0704	acc=0.9759	527.0 examples/second
[Step=100]	Loss=0.0677	acc=0.9770	4827.9 examples/second
[Step=150]	Loss=0.0690	acc

In [13]:
# Check sparsity of the final model, make sure it's 80%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here: can copy from previous question
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np_weight.size - np.count_nonzero(np_weight)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))
        
        
test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.4.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv1.0.conv: 0.8000217013888888
Sparsity of body_op.5.conv2.0.conv: 0.8000217013888888
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (e) Global iterative pruning

In [8]:
def global_prune_by_percentage(net, q=80.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            np_weight = layer.weight.detach().cpu().numpy()
            # Flatten the weight and append to flattened_weights
            flattened_weights.append(np_weight.flatten())
    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    flattened_weights_abs = np.absolute(flattened_weights)
    thres = np.percentile(flattened_weights_abs, q)
    
    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            np_weight = layer.weight.detach().cpu().numpy()
            # Generate a binary mask same shape as weight to decide which element to prune
            weight_abs = np.absolute(np_weight)
            binary_mask = weight_abs > thres
            # Convert mask to torch tensor and put on GPU
            mask_tensor = torch.from_numpy(binary_mask).float().to(device)
            # Multiply the weight by mask to perform pruning
            layer.weight.data = layer.weight.data * mask_tensor
            


In [15]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    q=(epoch+1)*8
    
    net.train()
    # Increase model sparsity
    if epoch<10:
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0516	acc=0.9830	539.8 examples/second
[Step=100]	Loss=0.0508	acc=0.9833	4835.6 examples/second
[Step=150]	Loss=0.0503	acc=0.9838	4834.5 examples/second
Test Loss=0.3227, Test acc=0.9144

Epoch: 1
[Step=50]	Loss=0.0471	acc=0.9847	540.4 examples/second
[Step=100]	Loss=0.0470	acc=0.9838	4839.2 examples/second
[Step=150]	Loss=0.0477	acc=0.9843	4807.8 examples/second
Test Loss=0.3244, Test acc=0.9147

Epoch: 2
[Step=50]	Loss=0.0480	acc=0.9852	519.7 examples/second
[Step=100]	Loss=0.0481	acc=0.9854	4833.1 examples/second
[Step=150]	Loss=0.0493	acc=0.9845	4819.4 examples/second
Test Loss=0.3247, Test acc=0.9140

Epoch: 3
[Step=50]	Loss=0.0546	acc=0.9822	540.2 examples/second
[Step=100]	Loss=0.0536	acc=0.9829	4820.8 examples/second
[Step=150]	Loss=0.0539	acc=0.9828	4837.8 examples/second
Test Loss=0.3260, Test acc=0.9143

Epoch: 4
[Step=50]	Loss=0.0592	acc=0.9799	559.8 examples/second
[Step=100]	Loss=0.0609	acc=0.9801	4837.9 examples/second
[Step=150]	Loss=0.0606	acc

In [16]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = np_weight.size - np.count_nonzero(np_weight)
        # Count number of parameters
        total = np_weight.size
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.3101851851851852
Sparsity of body_op.0.conv1.0.conv: 0.6584201388888888
Sparsity of body_op.0.conv2.0.conv: 0.6380208333333334
Sparsity of body_op.1.conv1.0.conv: 0.6263020833333334
Sparsity of body_op.1.conv2.0.conv: 0.6484375
Sparsity of body_op.2.conv1.0.conv: 0.6315104166666666
Sparsity of body_op.2.conv2.0.conv: 0.6684027777777778
Sparsity of body_op.3.conv1.0.conv: 0.6234809027777778
Sparsity of body_op.3.conv2.0.conv: 0.6884765625
Sparsity of body_op.4.conv1.0.conv: 0.7259114583333334
Sparsity of body_op.4.conv2.0.conv: 0.7829861111111112
Sparsity of body_op.5.conv1.0.conv: 0.7250434027777778
Sparsity of body_op.5.conv2.0.conv: 0.8133680555555556
Sparsity of body_op.6.conv1.0.conv: 0.7322591145833334
Sparsity of body_op.6.conv2.0.conv: 0.7645670572916666
Sparsity of body_op.7.conv1.0.conv: 0.7769911024305556
Sparsity of body_op.7.conv2.0.conv: 0.8260633680555556
Sparsity of body_op.8.conv1.0.conv: 0.8526204427083334
Sparsity of body_op.8.conv2.0.c

### Lab 3 (b) and (c): Fixed-point quantization

In [30]:
# Define quantized model and load weight
#Nbits = 6 #Change this value to finish (b) and (c)
#Nbits = 5
#Nbits = 4
#Nbits = 3
Nbits = 2

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)


Files already downloaded and verified
Test Loss=9.5441, Test accuracy=0.0899


In [31]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=1.3516	acc=0.6209	553.7 examples/second
[Step=100]	Loss=1.0775	acc=0.6842	4752.7 examples/second
[Step=150]	Loss=0.9499	acc=0.7137	4736.4 examples/second
Test Loss=0.8030, Test acc=0.7636
Saving...

Epoch: 1
[Step=200]	Loss=0.5760	acc=0.8145	484.2 examples/second
[Step=250]	Loss=0.5728	acc=0.8101	4758.7 examples/second
[Step=300]	Loss=0.5673	acc=0.8090	4740.5 examples/second
[Step=350]	Loss=0.5496	acc=0.8144	4746.9 examples/second
Test Loss=0.6345, Test acc=0.8066
Saving...

Epoch: 2
[Step=400]	Loss=0.4651	acc=0.8438	491.0 examples/second
[Step=450]	Loss=0.4782	acc=0.8332	4763.4 examples/second
[Step=500]	Loss=0.4748	acc=0.8366	4770.4 examples/second
[Step=550]	Loss=0.4653	acc=0.8394	4762.2 examples/second
Test Loss=0.6550, Test acc=0.8078
Saving...

Epoch: 3
[Step=600]	Loss=0.4282	acc=0.8503	485.5 examples/second
[Step=650]	Loss=0.4315	acc=0.8515	4762.1 examples/se

### Lab3 (d) Quantize pruned model

In [38]:
# Define quantized model and load weight
#Nbits = 4 #Change this value to finish (d)
#Nbits = 3
Nbits = 2

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)


Files already downloaded and verified
Test Loss=7.1126, Test accuracy=0.1026


In [39]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified

Epoch: 0
[Step=50]	Loss=2.3067	acc=0.1448	526.2 examples/second
[Step=100]	Loss=2.2477	acc=0.1556	4769.3 examples/second
[Step=150]	Loss=2.2232	acc=0.1618	4745.9 examples/second
Test Loss=2.1521, Test acc=0.1685
Saving...

Epoch: 1
[Step=200]	Loss=2.1340	acc=0.1865	466.3 examples/second
[Step=250]	Loss=2.1438	acc=0.1861	4737.6 examples/second
[Step=300]	Loss=2.1393	acc=0.1899	4754.3 examples/second
[Step=350]	Loss=2.1316	acc=0.1978	4760.6 examples/second
Test Loss=2.1157, Test acc=0.2055
Saving...

Epoch: 2
[Step=400]	Loss=2.1095	acc=0.2129	467.4 examples/second
[Step=450]	Loss=2.1014	acc=0.2201	4773.0 examples/second
[Step=500]	Loss=2.1010	acc=0.2207	4763.3 examples/second
[Step=550]	Loss=2.0972	acc=0.2229	4755.7 examples/second
Test Loss=2.0938, Test acc=0.2316
Saving...

Epoch: 3
[Step=600]	Loss=2.0955	acc=0.2292	477.2 examples/second
[Step=650]	Loss=2.0799	acc=0.2315	4755.5 examples/se

### Lab3 (e) Prune quantized model

In [9]:
net = ResNetCIFAR(num_layers=20, Nbits=4)
net = net.to(device)

net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    q=(epoch+1)*8
    
    net.train()
    # Increase model sparsity
    if epoch<10:
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "quantized_net_after_global_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0627	acc=0.9788	526.0 examples/second
[Step=100]	Loss=0.0647	acc=0.9784	4842.0 examples/second
[Step=150]	Loss=0.0661	acc=0.9777	4820.4 examples/second
Test Loss=0.3388, Test acc=0.9083

Epoch: 1
[Step=50]	Loss=0.0714	acc=0.9727	547.6 examples/second
[Step=100]	Loss=0.0688	acc=0.9751	4858.8 examples/second
[Step=150]	Loss=0.0693	acc=0.9752	4847.4 examples/second
Test Loss=0.3446, Test acc=0.9080

Epoch: 2
[Step=50]	Loss=0.0709	acc=0.9758	552.1 examples/second
[Step=100]	Loss=0.0706	acc=0.9754	4853.7 examples/second
[Step=150]	Loss=0.0706	acc=0.9756	4829.8 examples/second
Test Loss=0.3470, Test acc=0.9067

Epoch: 3
[Step=50]	Loss=0.0773	acc=0.9724	548.9 examples/second
[Step=100]	Loss=0.0755	acc=0.9735	4832.0 examples/second
[Step=150]	Loss=0.0755	acc=0.9738	4846.3 examples/second
Test Loss=0.3469, Test acc=0.9067

Epoch: 4
[Step=50]	Loss=0.0781	acc=0.9730	557.0 examples/second
[Step=100]	Loss=0.0825	acc=0.9712	4838.9 examples/second
[Step=150]	Loss=0.0837	acc

In [10]:
net.load_state_dict(torch.load("quantized_net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        np_weight = layer.weight.data.cpu().numpy()
        zeros = (np_weight==0).sum()
        total = np_weight.flatten().shape[0]
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.30787037037037035
Sparsity of body_op.0.conv1.0.conv: 0.6575520833333334
Sparsity of body_op.0.conv2.0.conv: 0.6397569444444444
Sparsity of body_op.1.conv1.0.conv: 0.6280381944444444
Sparsity of body_op.1.conv2.0.conv: 0.6488715277777778
Sparsity of body_op.2.conv1.0.conv: 0.6315104166666666
Sparsity of body_op.2.conv2.0.conv: 0.6684027777777778
Sparsity of body_op.3.conv1.0.conv: 0.6252170138888888
Sparsity of body_op.3.conv2.0.conv: 0.6882595486111112
Sparsity of body_op.4.conv1.0.conv: 0.7261284722222222
Sparsity of body_op.4.conv2.0.conv: 0.7820095486111112
Sparsity of body_op.5.conv1.0.conv: 0.7249348958333334
Sparsity of body_op.5.conv2.0.conv: 0.8132595486111112
Sparsity of body_op.6.conv1.0.conv: 0.7331814236111112
Sparsity of body_op.6.conv2.0.conv: 0.7640245225694444
Sparsity of body_op.7.conv1.0.conv: 0.7770724826388888
Sparsity of body_op.7.conv2.0.conv: 0.8261176215277778
Sparsity of body_op.8.conv1.0.conv: 0.8527018229166666
Sparsity of bod